In [11]:
# %pip install numpy
# %pip install pandas
# %pip install torch

In [12]:
#installing the required libraries
# %pip install transformers
# %pip install torchtext
# %pip install sentencepiece
# %pip install datasets
# %pip install torchmetrics
# %pip install matplotlib
# %pip install seaborn
# %pip install scikit-learn
# #huggiface download
# %pip install huggingface_hub


In [13]:
# %pip install llama-cpp-python

In [14]:
from huggingface_hub import hf_hub_download

In [15]:
from llama_cpp import Llama

In [16]:
#download the model llama 2 13b chat gguf
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_S.gguf" # the model is in gguf format

In [17]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

#print toal path
print(model_path)

/home/biomedialab/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_S.gguf


In [18]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )
     

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /home/biomedialab/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rop

In [22]:
prompt= f"Is 7 composite?"

prompt_template=f'''You are a math assistant. I will ask you to find wheather given number is composite or not. Please answer in the correct format. For example, if I ask 'Is 5 composite?' , you should answer 'Yes' if 5 is a composite number or 'No' if 5 is not a composite number. Do not include any other information in your answer.
USER: {prompt}

ASSISTANT:'''
        

In [23]:
response=lcpp_llm(prompt=prompt_template, max_tokens=500, temperature=0.00001, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=False)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4490.96 ms
llama_print_timings:      sample time =       0.82 ms /     2 runs   (    0.41 ms per token,  2450.98 tokens per second)
llama_print_timings: prompt eval time =     504.02 ms /     9 tokens (   56.00 ms per token,    17.86 tokens per second)
llama_print_timings:        eval time =     675.01 ms /     1 runs   (  675.01 ms per token,     1.48 tokens per second)
llama_print_timings:       total time =    1184.77 ms /    10 tokens


In [24]:
print(response["choices"][0]["text"])

 Yes


In [25]:
import json

In [29]:
#load file in data folder int_addition.json
with open('primality_test.json') as f:
    data = json.load(f)

In [30]:
#data as pandas dataframe
import pandas as pd
df = pd.DataFrame(data)
#last 5 rows
df

,primes,composites
0,3187,99160
1,59509,12005
2,58787,33309
3,22861,73881
4,89519,78681
...,...,...
4995,38053,83445
4996,74797,35077
4997,78823,18386
4998,61751,19887


In [31]:
df.iloc[1, 0]

59509

In [32]:
import time


for i in range(0, len(df)):
    
    if i%100==0:
        print(f"Completed {i} rows")
        
    try:
        
        prompt= f"Is {df.iloc[i, 1]} composite?"
        
        prompt_template=f'''You are a math assistant. I will ask you to find wheather given number is composite or not. Please answer in the correct format. For example, if I ask 'Is 5 composite?' , you should answer 'Yes' if 5 is a composite number or 'No' if 5 is not a composite number. Do not include any other information in your answer.

        USER: {prompt}
        
        ASSISTANT:'''
        
        response=lcpp_llm(prompt=prompt_template, max_tokens=500, temperature=0.00001, top_p=0.95,
                    repeat_penalty=1.2, top_k=150,
                    echo=False)
        
        with open('composite_test_prime.txt', 'a') as f:
            answer=response["choices"][0]["text"]
            f.write(f"{df.iloc[i, 0]} = {answer} \n")
            
            
    except:
        
        with open('composite_test_prime.txt', 'a') as f:
            f.write(f"{df.iloc[i, 0]} = Error \n")
            
        time.sleep(2)

Completed 0 rows


Llama.generate: prefix-match hit

llama_print_timings:        load time =    4490.96 ms
llama_print_timings:      sample time =       0.82 ms /     2 runs   (    0.41 ms per token,  2427.18 tokens per second)
llama_print_timings: prompt eval time =    1196.40 ms /    23 tokens (   52.02 ms per token,    19.22 tokens per second)
llama_print_timings:        eval time =     716.87 ms /     1 runs   (  716.87 ms per token,     1.39 tokens per second)
llama_print_timings:       total time =    1919.36 ms /    24 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    4490.96 ms
llama_print_timings:      sample time =       0.77 ms /     2 runs   (    0.38 ms per token,  2600.78 tokens per second)
llama_print_timings: prompt eval time =     852.77 ms /    16 tokens (   53.30 ms per token,    18.76 tokens per second)
llama_print_timings:        eval time =     701.90 ms /     1 runs   (  701.90 ms per token,     1.42 tokens per second)
llama_print_timings:       to

Completed 100 rows



llama_print_timings:        load time =    4490.96 ms
llama_print_timings:      sample time =       0.83 ms /     2 runs   (    0.41 ms per token,  2421.31 tokens per second)
llama_print_timings: prompt eval time =     810.77 ms /    16 tokens (   50.67 ms per token,    19.73 tokens per second)
llama_print_timings:        eval time =     666.50 ms /     1 runs   (  666.50 ms per token,     1.50 tokens per second)
llama_print_timings:       total time =    1483.00 ms /    17 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    4490.96 ms
llama_print_timings:      sample time =       0.84 ms /     2 runs   (    0.42 ms per token,  2389.49 tokens per second)
llama_print_timings: prompt eval time =     842.39 ms /    16 tokens (   52.65 ms per token,    18.99 tokens per second)
llama_print_timings:        eval time =     713.69 ms /     1 runs   (  713.69 ms per token,     1.40 tokens per second)
llama_print_timings:       total time =    1561.69 ms /    17 

Completed 200 rows


...


llama_print_timings:        load time =    4490.96 ms
llama_print_timings:      sample time =       0.88 ms /     2 runs   (    0.44 ms per token,  2270.15 tokens per second)
llama_print_timings: prompt eval time =     824.94 ms /    16 tokens (   51.56 ms per token,    19.40 tokens per second)
llama_print_timings:        eval time =     660.82 ms /     1 runs   (  660.82 ms per token,     1.51 tokens per second)
llama_print_timings:       total time =    1491.46 ms /    17 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    4490.96 ms
llama_print_timings:      sample time =       0.81 ms /     2 runs   (    0.40 ms per token,  2481.39 tokens per second)
llama_print_timings: prompt eval time =     841.72 ms /    16 tokens (   52.61 ms per token,    19.01 tokens per second)
llama_print_timings:        eval time =     666.39 ms /     1 runs   (  666.39 ms per token,     1.50 tokens per second)
llama_print_timings:       total time =    1514.57 ms /    17 

Completed 4800 rows



llama_print_timings:        load time =    4490.96 ms
llama_print_timings:      sample time =       0.78 ms /     2 runs   (    0.39 ms per token,  2570.69 tokens per second)
llama_print_timings: prompt eval time =     843.55 ms /    16 tokens (   52.72 ms per token,    18.97 tokens per second)
llama_print_timings:        eval time =     708.51 ms /     1 runs   (  708.51 ms per token,     1.41 tokens per second)
llama_print_timings:       total time =    1557.92 ms /    17 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    4490.96 ms
llama_print_timings:      sample time =       0.79 ms /     2 runs   (    0.40 ms per token,  2531.65 tokens per second)
llama_print_timings: prompt eval time =     810.57 ms /    16 tokens (   50.66 ms per token,    19.74 tokens per second)
llama_print_timings:        eval time =     659.92 ms /     1 runs   (  659.92 ms per token,     1.52 tokens per second)
llama_print_timings:       total time =    1475.96 ms /    17 

Completed 4900 rows



llama_print_timings:        load time =    4490.96 ms
llama_print_timings:      sample time =       0.80 ms /     2 runs   (    0.40 ms per token,  2512.56 tokens per second)
llama_print_timings: prompt eval time =     815.88 ms /    16 tokens (   50.99 ms per token,    19.61 tokens per second)
llama_print_timings:        eval time =     651.52 ms /     1 runs   (  651.52 ms per token,     1.53 tokens per second)
llama_print_timings:       total time =    1473.61 ms /    17 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    4490.96 ms
llama_print_timings:      sample time =       0.85 ms /     2 runs   (    0.42 ms per token,  2355.71 tokens per second)
llama_print_timings: prompt eval time =     826.19 ms /    16 tokens (   51.64 ms per token,    19.37 tokens per second)
llama_print_timings:        eval time =     659.51 ms /     1 runs   (  659.51 ms per token,     1.52 tokens per second)
llama_print_timings:       total time =    1491.81 ms /    17 